<a href="https://colab.research.google.com/github/rayssabanti/ia-fine-tuning-law/blob/main/ia_fine_tuning_eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Applied to Law

## Goal
Perform Fine-Tuning of a language model to make text predictions from Web Scraping of legal texts.

In [71]:
from platform import python_version

## Pips

In [2]:
!pip install -q -U watermark #versionamento de dependencias

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.3 MB/s eta 0:00:00


In [3]:
!pip install -q torch

In [4]:
!pip install -q transformers==4.32.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.5 MB/s eta 0:00:00


In [5]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00


### Imports

In [72]:
import re
import bs4 #pacote python para raspagem de dados
import torch #modelo prétreinado
import random
import requests
import transformers #hugging
from tqdm import tqdm # barra de progressão
from bs4 import BeautifulSoup
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset
from transformers import AutoModelForMaskedLM, AutoTokenizer, BertForQuestionAnswering
from transformers import pipeline

### Dependências

In [7]:
%reload_ext watermark
%watermark -a "Ray" --iversions


Author: Ray

requests    : 2.31.0
transformers: 4.32.1
torch       : 2.0.1+cu118
re          : 2.2.1
bs4         : 4.11.2



## Device

Hardware used to run the code:

In [8]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [9]:
if torch.cuda.is_available():
    print('GPU:', torch.cuda.device_count())
    print('GPU Model:',torch.cuda.get_device_name(0))
    print('GPU Memory:',torch.cuda.get_device_properties(0).total_memory/1e9)

GPU: 1
GPU Model: Tesla V100-SXM2-16GB
GPU Memory: 16.935419904


## Web Scraping of Legal Texts

check robots.txt ==   https://www.uol.com.br/robots.txt







In [11]:
# Scraping
# List
textos_juridicos = []

# Loop
for url in ["https://en.wikipedia.org/wiki/Justice",
            "https://en.wikipedia.org/wiki/Sentence_(law)",
            "https://en.wikipedia.org/wiki/Law",
            "https://en.wikipedia.org/wiki/Federation",
            "https://en.wikipedia.org/wiki/Trial"]:

    resp = requests.get(url)

    soup = BeautifulSoup(resp.text, 'html.parser')

    for para in soup.find_all('p'):
        textos_juridicos.append(para.text)

In [12]:
docs = textos_juridicos

In [13]:
docs

['\n',
 'Justice, in its broadest sense, is the concept that individuals are to be treated in a manner that is equitable and fair.[1]\n',
 'To achieve justice, individuals should receive that which they deserve, with the interpretation of what "deserve" means, in turn, drawing on numerous viewpoints and perspectives, including fields like ethics, rationality, law, religion, equity and fairness. The state may be said to pursue justice by operating courts and enforcing their rulings.\n',
 'A variety of philosophical and moral theories have been advanced to inform understanding of justice.\n',
 'Early theories of justice were set out by the Ancient Greek philosophers Plato, in his work The Republic, and Aristotle, in his Nicomachean Ethics and Politics. \n',
 'Religious explanations of the justice can be grouped under divine command theory, which holds that justice issues from God.[2]\n',
 'Western thinkers later advanced different theories of where foundations of justice lie.  In the 17t

## Generating the files



*We have to shuffle the data to learn only from context.*




In [14]:
random.shuffle(docs)

In [15]:
len(docs)

220

In [16]:
#saving
with open("docs.txt", "w") as f:
    f.write("\n".join(docs))

## Modeling and Tokenization


ref = https://huggingface.co/bert-base-uncased



In [17]:
%env TF_ENABLE_ONEDNN_OPTS=0

env: TF_ENABLE_ONEDNN_OPTS=0


By default, tensorflow sees if the environment is optimized, this silences it.

### Model Imports

In [73]:
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [75]:

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


### Tokenization





In [20]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding = "max_length", truncation = True)

In [76]:
try:
    with open('docs.txt', 'r') as file:
        data = file.readlines()
    dataset = Dataset.from_dict({'text': data})
except FileNotFoundError:
    print("The file was not found.")

In [23]:
tokenized_dataset = dataset.map(tokenize_function, batched = True, num_proc = 4, remove_columns = ["text"])

Map (num_proc=4):   0%|          | 0/433 [00:00<?, ? examples/s]

#### Formatting the data

In [24]:
tokenized_dataset.set_format(type = 'torch', columns = ['input_ids', 'attention_mask'])

In [25]:
tokenized_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 433
})

#### DataLoader

In [26]:
dl_docs = DataLoader(tokenized_dataset, shuffle = True, batch_size = 8)

#### Optimizer

In [27]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 5e-5)

#### Structure

In [28]:
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

## Fine-Tunning

In [29]:
model.train() #mode train

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [30]:
num_epochs = 10
gradient_accumulation_steps = 4 #mostra durante o modelo

In [31]:
%%time

# Loop through the number of epochs for training
for epoch in range(num_epochs):

    # Enumerates each dataloader batch
    for step, batch in enumerate(dl_docs):

        # Moves inputs and attention masks to the device
        inputs, attention_mask = batch['input_ids'].to(device), batch['attention_mask'].to(device)

        # Passage to the front of the model to get the exits
        outputs = model(inputs, attention_mask = attention_mask, labels = inputs)

        # Extracts loss from outputs
        loss = outputs.loss

        # Normalizes the loss along the gradient_accumulation_steps
        loss = loss / gradient_accumulation_steps

        print("Erro:", loss)

        loss.backward()

        # Update the weights
        if (step + 1) % gradient_accumulation_steps == 0 or step == len(dl_docs) - 1:

            # optimization
            optimizer.step()


            optimizer.zero_grad()

Erro: tensor(4.5362, device='cuda:0', grad_fn=<DivBackward0>)
Erro: tensor(3.1241, device='cuda:0', grad_fn=<DivBackward0>)
Erro: tensor(4.4094, device='cuda:0', grad_fn=<DivBackward0>)
Erro: tensor(4.3269, device='cuda:0', grad_fn=<DivBackward0>)
Erro: tensor(3.3578, device='cuda:0', grad_fn=<DivBackward0>)
Erro: tensor(3.2705, device='cuda:0', grad_fn=<DivBackward0>)
Erro: tensor(3.2720, device='cuda:0', grad_fn=<DivBackward0>)
Erro: tensor(3.3157, device='cuda:0', grad_fn=<DivBackward0>)
Erro: tensor(2.5179, device='cuda:0', grad_fn=<DivBackward0>)
Erro: tensor(2.8141, device='cuda:0', grad_fn=<DivBackward0>)
Erro: tensor(2.5441, device='cuda:0', grad_fn=<DivBackward0>)
Erro: tensor(2.6536, device='cuda:0', grad_fn=<DivBackward0>)
Erro: tensor(2.4959, device='cuda:0', grad_fn=<DivBackward0>)
Erro: tensor(2.5369, device='cuda:0', grad_fn=<DivBackward0>)
Erro: tensor(2.4667, device='cuda:0', grad_fn=<DivBackward0>)
Erro: tensor(2.2543, device='cuda:0', grad_fn=<DivBackward0>)
Erro: te

In [32]:
# Saving
model.save_pretrained("ia_bert_model")
tokenizer.save_pretrained("ia_bert_tokenizer")

('ia_bert_tokenizer/tokenizer_config.json',
 'ia_bert_tokenizer/special_tokens_map.json',
 'ia_bert_tokenizer/vocab.txt',
 'ia_bert_tokenizer/added_tokens.json',
 'ia_bert_tokenizer/tokenizer.json')



## Predict

#### custom tokenizer


In [34]:
tokenizer_dsa = AutoTokenizer.from_pretrained("ia_bert_tokenizer")

In [36]:
model_dsa = AutoModelForMaskedLM.from_pretrained("ia_bert_model")

In [37]:
# model to device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_dsa = model_dsa.to(device)

### checking if the model can predict the words

In [77]:
#NEW DATA
texto = "In law, a trial is a coming together of parties to a dispute, to present information in a tribunal"

In [46]:

texto_tokenizado = tokenizer_dsa(texto, return_tensors = 'pt')

In [47]:
texto_tokenizado

{'input_ids': tensor([[  101,  1999,  2375,  1010,  1037,  3979,  2003,  1037,  2746,  2362,
          1997,  4243,  2000,  1037,  7593,  1010,  2000,  2556,  2592,  1999,
          1037, 12152,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [70]:
texto_tokenizado = texto_tokenizado.to(device)

In [49]:
# predict
with torch.no_grad():
    output = model(**texto_tokenizado)

In [52]:
output

MaskedLMOutput(loss=None, logits=tensor([[[  5.0513,  -3.4150,  -3.3520,  ...,  -3.1506,  -5.8751,  -3.2925],
         [ -6.6862, -10.9693, -11.4006,  ..., -10.7455, -12.0321, -13.3961],
         [  1.4822,  -4.5520,  -4.7655,  ...,  -5.0719,  -5.6323,  -5.7723],
         ...,
         [ -2.8708,  -7.6943,  -7.7677,  ...,  -8.2252,  -7.4922,  -5.8579],
         [ 12.2133,   4.7825,   4.2225,  ...,   2.2323,   1.3759,   3.2667],
         [ -8.1690, -10.1962, -10.5770,  ...,  -8.1967,  -8.9926,  -9.2241]]],
       device='cuda:0'), hidden_states=None, attentions=None)

In [53]:
# trad. output
predicted_token_ids = torch.argmax(output.logits, dim = -1)


argmax == takes the largest value

In [54]:
# Token ids
predicted_token_ids

tensor([[  101,  1999,  2375,  1010,  1037,  3979,  2003,  1037,  2746,  2362,
          1997,  4243,  2000,  1037,  7593,  1010,  2000,  2556,  2592,  1999,
          1037, 12152,  1012]], device='cuda:0')

In [55]:
# Decode
predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_token_ids[0])

In [56]:
# Tokens previstos
predicted_tokens

['[CLS]',
 'in',
 'law',
 ',',
 'a',
 'trial',
 'is',
 'a',
 'coming',
 'together',
 'of',
 'parties',
 'to',
 'a',
 'dispute',
 ',',
 'to',
 'present',
 'information',
 'in',
 'a',
 'tribunal',
 '.']

In [57]:
print(predicted_tokens)

['[CLS]', 'in', 'law', ',', 'a', 'trial', 'is', 'a', 'coming', 'together', 'of', 'parties', 'to', 'a', 'dispute', ',', 'to', 'present', 'information', 'in', 'a', 'tribunal', '.']


## Predict missing



*   https://huggingface.co/docs/transformers/main_classes/pipelines




In [59]:
# text = "In law, a trial is a coming together of parties to a dispute, to present information in a tribunal"
texto = "In law, a trial is a coming together of parties to a [MASK], to present information in a tribunal"

In [60]:
#pipeline
unmasker_pipeline = pipeline('fill-mask', model = model_dsa, tokenizer = tokenizer_dsa, device = 0)

In [61]:

unmasker_pipeline(texto)

[{'score': 0.37123581767082214,
  'token': 0,
  'token_str': '[PAD]',
  'sequence': 'in law, a trial is a coming together of parties to a, to present information in a tribunal'},
 {'score': 0.10854265093803406,
  'token': 7593,
  'token_str': 'dispute',
  'sequence': 'in law, a trial is a coming together of parties to a dispute, to present information in a tribunal'},
 {'score': 0.027915243059396744,
  'token': 16077,
  'token_str': 'covenant',
  'sequence': 'in law, a trial is a coming together of parties to a covenant, to present information in a tribunal'},
 {'score': 0.026693519204854965,
  'token': 3206,
  'token_str': 'contract',
  'sequence': 'in law, a trial is a coming together of parties to a contract, to present information in a tribunal'},
 {'score': 0.01868905872106552,
  'token': 9870,
  'token_str': 'lawsuit',
  'sequence': 'in law, a trial is a coming together of parties to a lawsuit, to present information in a tribunal'}]